# Simple baseline

Simple and very small (!) model trained on the real train dataset for this competition: 
https://www.kaggle.com/datasets/thedrcat/daigt-proper-train-dataset

**I'd appreciate if you can upvote both the dataset and the notebook if you find it helpful! 🙏❤️**

I'm planning to keep maintaining the train dataset with more data and to publish a proper instrumented training/inference notebooks with larger models (coming soon!). 

In [ ]:
import transformers
import datasets
import pandas as pd
import numpy as np

In [ ]:
model_checkpoint = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall"

In [ ]:
from datasets import Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')

In [ ]:
train = df[df.fold != 0].reset_index(drop=True)
valid = df[df.fold == 0].reset_index(drop=True)
train.head()

In [ ]:
# Let's try to undersample the persuade_corpus:
not_persuade_df = train[train['source'] != 'persuade_corpus']
persuade_df = train[train['source'] == 'persuade_corpus']
sampled_persuade_df = persuade_df.sample(n=6000, random_state=42)
train = pd.concat([not_persuade_df, sampled_persuade_df]).sample(frac=1, random_state=42).reset_index(drop=True)
train.source.value_counts()

In [ ]:
ds_train = Dataset.from_pandas(train)
ds_valid = Dataset.from_pandas(valid)

In [ ]:
ds_train

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=128, padding=True, truncation=True)

In [ ]:
ds_train_enc = ds_train.map(preprocess_function, batched=True)

In [ ]:
ds_valid_enc = ds_valid.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
metric_name = "roc_auc"
model_name = "deberta-xsmall"
train_batch_size = 4
eval_batch_size = 32
grad_acc = 4

args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 246,
    save_steps = 246,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    report_to='none', # change to wandb after enabling internet access
)

In [ ]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_train_enc,
    eval_dataset=ds_valid_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_ds = Dataset.from_pandas(test)
test_ds_enc = test_ds.map(preprocess_function, batched=True)

In [ ]:
test_preds = trainer.predict(test_ds_enc)

In [ ]:
logits = test_preds.predictions
probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
sub = pd.DataFrame()
sub['id'] = test['id']
sub['generated'] = probs[:,1]
sub.to_csv('submission.csv', index=False)
sub.head()

In [ ]:
test2 = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
test2['label'] = test2['generated']
test_ds2 = Dataset.from_pandas(test2)
test_ds_enc2 = test_ds2.map(preprocess_function, batched=True)

In [ ]:
trainer.evaluate(test_ds_enc2)